<a href="https://colab.research.google.com/github/prakharsrivastava/powerbi/blob/main/Data_Science_on_Google_Cloud_Platform_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/chapter2/


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/chapter2


In [55]:
import pandas as pd
df = pd.read_csv('airports.csv',low_memory=False)
df.head()

,AIRPORT_SEQ_ID,AIRPORT_ID,AIRPORT,DISPLAY_AIRPORT_NAME,DISPLAY_AIRPORT_CITY_NAME_FULL,AIRPORT_WAC_SEQ_ID2,AIRPORT_WAC,AIRPORT_COUNTRY_NAME,AIRPORT_COUNTRY_CODE_ISO,AIRPORT_STATE_NAME,...,LON_HEMISPHERE,LON_MINUTES,LON_SECONDS,LONGITUDE,UTC_LOCAL_TIME_VARIATION,AIRPORT_START_DATE,AIRPORT_THRU_DATE,AIRPORT_IS_CLOSED,AIRPORT_IS_LATEST,Unnamed: 32
0,1000101,10001,01A,Afognak Lake Airport,"Afognak Lake, AK",101,1,United States,US,Alaska,...,W,54.0,24.0,-152.906667,NaN,2007-07-01,NaN,0,1,NaN
1,1000301,10003,03A,Bear Creek Mining Strip,"Granite Mountain, AK",101,1,United States,US,Alaska,...,W,4.0,18.0,-161.071667,NaN,2007-07-01,NaN,0,1,NaN
2,1000401,10004,04A,Lik Mining Camp,"Lik, AK",101,1,United States,US,Alaska,...,W,10.0,0.0,-163.166667,NaN,2007-07-01,NaN,0,1,NaN
3,1000501,10005,05A,Little Squaw Airport,"Little Squaw, AK",101,1,United States,US,Alaska,...,W,11.0,2.0,-148.183889,NaN,2007-08-01,NaN,0,1,NaN
4,1000601,10006,06A,Kizhuyak Bay,"Kizhuyak, AK",101,1,United States,US,Alaska,...,W,52.0,58.0,-152.882778,NaN,2007-10-01,NaN,0,1,NaN


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
df["LONGITUDE"]

0       -152.906667
1       -161.071667
2       -163.166667
3       -148.183889
4       -152.882778
            ...    
13381    -72.007778
13382    -71.478611
13383     19.398056
13384    -96.071111
13385           NaN
Name: LONGITUDE, Length: 13386, dtype: float64

ValueError: ignored

In [27]:
!pip install apache-beam[gcp]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.2/526.2 KB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.7/140.7 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 KB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.9/273.9 KB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.7/268.7 KB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.2/148.2 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 KB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.4/185.

In [8]:

import apache_beam as beam
import csv
if __name__ == '__main__':
    with beam.Pipeline('DirectRunner') as pipeline:
        airports = (pipeline
                    | 'airports:read' >> beam.io.ReadFromText('airports.csv')
                    | beam.Filter(lambda line: "United States" in line)
                    | 'airports:fields' >> beam.Map(lambda line: next(csv.reader([line])))
                    | 'airports:tz' >> beam.Map(lambda fields: (fields[0], addtimezone(fields[21], fields[26])))
                    )

        flights = (pipeline
                   | 'flights:read' >> beam.io.ReadFromText('flights_sample.json')
                   | 'flights:tzcorr' >> beam.FlatMap(tz_correct, beam.pvalue.AsDict(airports))
                   )

        flights | beam.io.textio.WriteToText('all_flights')

NameError: ignored

In [17]:
df.head()

,AIRPORT_SEQ_ID,AIRPORT_ID,AIRPORT,DISPLAY_AIRPORT_NAME,DISPLAY_AIRPORT_CITY_NAME_FULL,AIRPORT_WAC_SEQ_ID2,AIRPORT_WAC,AIRPORT_COUNTRY_NAME,AIRPORT_COUNTRY_CODE_ISO,AIRPORT_STATE_NAME,...,LON_HEMISPHERE,LON_MINUTES,LON_SECONDS,LONGITUDE,UTC_LOCAL_TIME_VARIATION,AIRPORT_START_DATE,AIRPORT_THRU_DATE,AIRPORT_IS_CLOSED,AIRPORT_IS_LATEST,Unnamed: 32
0,1000101,10001,01A,Afognak Lake Airport,"Afognak Lake, AK",101,1,United States,US,Alaska,...,W,54.0,24.0,-152.906667,NaN,2007-07-01,NaN,0,1,NaN
1,1000301,10003,03A,Bear Creek Mining Strip,"Granite Mountain, AK",101,1,United States,US,Alaska,...,W,4.0,18.0,-161.071667,NaN,2007-07-01,NaN,0,1,NaN
2,1000401,10004,04A,Lik Mining Camp,"Lik, AK",101,1,United States,US,Alaska,...,W,10.0,0.0,-163.166667,NaN,2007-07-01,NaN,0,1,NaN
3,1000501,10005,05A,Little Squaw Airport,"Little Squaw, AK",101,1,United States,US,Alaska,...,W,11.0,2.0,-148.183889,NaN,2007-08-01,NaN,0,1,NaN
4,1000601,10006,06A,Kizhuyak Bay,"Kizhuyak, AK",101,1,United States,US,Alaska,...,W,52.0,58.0,-152.882778,NaN,2007-10-01,NaN,0,1,NaN


In [10]:

import apache_beam as beam
import csv

if __name__ == '__main__':
    with beam.Pipeline('DirectRunner') as pipeline:
        airports = (pipeline
                    | beam.io.ReadFromText('airports.csv')
                    | beam.Map(lambda line: next(csv.reader([line])))
                    | beam.Map(lambda fields: (fields[0], (fields[21], fields[26])))
                    )

        (airports
         | beam.Map(lambda airport_data: '{},{}'.format(airport_data[0], ','.join(airport_data[1])))
         | beam.io.WriteToText('extracted_airports')
         )

In [11]:
import apache_beam as beam
import csv


def addtimezone(lat, lon):
    try:
        import timezonefinder
        tf = timezonefinder.TimezoneFinder()
        tz = tf.timezone_at(lng=float(lon), lat=float(lat))
        if tz is None:
            tz = 'UTC'
        return lat, lon, tz
    except ValueError:
        return lat, lon, 'TIMEZONE'  # header


if __name__ == '__main__':
    with beam.Pipeline('DirectRunner') as pipeline:
        airports = (pipeline
                    | beam.io.ReadFromText('airports.csv')
                    | beam.Filter(lambda line: "United States" in line)
                    | beam.Map(lambda line: next(csv.reader([line])))
                    | beam.Map(lambda fields: (fields[0], addtimezone(fields[21], fields[26])))
                    )

        airports | beam.Map(lambda f: '{},{}'.format(f[0], ','.join(f[1]))) | beam.io.textio.WriteToText(
            'airports_with_tz')

RuntimeError: ignored

In [14]:
from timezonefinder import TimezoneFinder
import pandas as pd

my_func = TimezoneFinder().timezone_at  #Note the no parenthesis on the function call!
df = pd.DataFrame({'latitude': [-22.540556,-22.950556,-22.967778], 'longitude': [-43.149167,-43.230833,-43.234444], 'timezone': [0,0,0]})
df['timezone'] = df.apply(lambda x: my_func(lng=x['longitude'], lat=x['latitude']),axis=1)
df

,latitude,longitude,timezone
0,-22.540556,-43.149167,America/Sao_Paulo
1,-22.950556,-43.230833,America/Sao_Paulo
2,-22.967778,-43.234444,America/Sao_Paulo


In [26]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 17.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824010 sha256=e85a7013765801fc94019167440243eb02fff4f2c180f7af391f603e8180e8b8
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [54]:
df.isna().sum()

latitude     0
longitude    0
timezone     0
dtype: int64

In [56]:
df['LATITUDE'].fillna(1, inplace=True)
df['LONGITUDE'].fillna(1, inplace=True)

In [59]:
from timezonefinder import TimezoneFinder
import pandas as pd

my_func = TimezoneFinder().timezone_at  #Note the no parenthesis on the function call!

df['TIMEZONE'] = df.apply(lambda x: my_func(lng=x['LONGITUDE'], lat=x['LATITUDE']),axis=1)
df

,AIRPORT_SEQ_ID,AIRPORT_ID,AIRPORT,DISPLAY_AIRPORT_NAME,DISPLAY_AIRPORT_CITY_NAME_FULL,AIRPORT_WAC_SEQ_ID2,AIRPORT_WAC,AIRPORT_COUNTRY_NAME,AIRPORT_COUNTRY_CODE_ISO,AIRPORT_STATE_NAME,...,LON_SECONDS,LONGITUDE,UTC_LOCAL_TIME_VARIATION,AIRPORT_START_DATE,AIRPORT_THRU_DATE,AIRPORT_IS_CLOSED,AIRPORT_IS_LATEST,Unnamed: 32,timezone,TIMEZONE
0,1000101,10001,01A,Afognak Lake Airport,"Afognak Lake, AK",101,1,United States,US,Alaska,...,24.0,-152.906667,NaN,2007-07-01,NaN,0,1,NaN,America/Anchorage,America/Anchorage
1,1000301,10003,03A,Bear Creek Mining Strip,"Granite Mountain, AK",101,1,United States,US,Alaska,...,18.0,-161.071667,NaN,2007-07-01,NaN,0,1,NaN,America/Anchorage,America/Anchorage
2,1000401,10004,04A,Lik Mining Camp,"Lik, AK",101,1,United States,US,Alaska,...,0.0,-163.166667,NaN,2007-07-01,NaN,0,1,NaN,America/Nome,America/Nome
3,1000501,10005,05A,Little Squaw Airport,"Little Squaw, AK",101,1,United States,US,Alaska,...,2.0,-148.183889,NaN,2007-08-01,NaN,0,1,NaN,America/Anchorage,America/Anchorage
4,1000601,10006,06A,Kizhuyak Bay,"Kizhuyak, AK",101,1,United States,US,Alaska,...,58.0,-152.882778,NaN,2007-10-01,NaN,0,1,NaN,America/Anchorage,America/Anchorage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13381,1673101,16731,NH3,Dean Memorial,"Haverhill, NH",1401,14,United States,US,New Hampshire,...,28.0,-72.007778,-500.0,2016-06-01,NaN,0,1,NaN,America/New_York,America/New_York
13382,1673201,16732,KNA,Vina del Mar Airport,"Vina del Mar, Chile",32401,324,Chile,CL,NaN,...,43.0,-71.478611,-400.0,2016-06-01,NaN,0,1,NaN,America/Santiago,America/Santiago
13383,1673301,16733,LCJ,Lodz Lublinek,"Lodz Lodzkie, Poland",46701,467,Poland,PL,NaN,...,53.0,19.398056,100.0,2016-06-01,NaN,0,1,NaN,Europe/Warsaw,Europe/Warsaw
13384,1673401,16734,NU4,Meadowbank,"Meadowbank Nunavut, Canada",93401,934,Canada,CA,Nunavut Territory,...,16.0,-96.071111,-500.0,2016-06-01,NaN,0,1,NaN,America/Rankin_Inlet,America/Rankin_Inlet
